In [6]:

!pip install dash
!pip install pandas

In [7]:
# Import packages
from dash import Dash, html, dash_table
import pandas as pd

# Incorporate data
df = pd.read_csv('analise-de-dados-clone.csv', low_memory=False)

# Check the schema of the data
print("Schema:")
print(df.info())  # Mostra informações do DataFrame, incluindo colunas e tipos de dados

# Identify potential errors
for col in df.columns:
    unique_types = df[col].map(type).unique()
    if len(unique_types) > 1:
        print(f"Column {col} has mixed types: {unique_types}")

# Initialize the app
app = Dash()

# App layout
app.layout = html.Div([
    html.H1(children='My First App with Data'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=10)
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30198 entries, 0 to 30197
Columns: 110 entries, ID do registro. to Associated Meeting IDs
dtypes: float64(14), int64(1), object(95)
memory usage: 25.3+ MB
None
Column Data de fechamento has mixed types: [<class 'str'> <class 'float'>]
Column Proprietário do negócio has mixed types: [<class 'str'> <class 'float'>]
Column Motivo de Perda has mixed types: [<class 'str'> <class 'float'>]
Column Sub-motivo de perda do negócio has mixed types: [<class 'str'> <class 'float'>]
Column Pré-análise de crédito has mixed types: [<class 'float'> <class 'str'>]
Column Tipo de Imóvel has mixed types: [<class 'str'> <class 'float'>]
Column Dados da fonte de tráfego mais recente 1 has mixed types: [<class 'str'> <class 'float'>]
Column Dados da fonte de tráfego mais recente 2 has mixed types: [<class 'float'> <class 'str'>]
Column Fonte de tráfego mais recente has mixed types: [<class 'str'> <class 'float'>]
Column Fonte original do tráfego has m

In [8]:
print(df.dtypes)


ID do registro.                            int64
Nome do negócio                           object
Etapa do negócio                          object
Data de fechamento                        object
Proprietário do negócio                   object
                                           ...  
Amora Fonte de conversao aprofundada 2    object
Amora Fonte original do tráfego           object
Amora Fonte original aprofundada 1        object
Amora Fonte original aprofundada 2        object
Associated Meeting IDs                    object
Length: 110, dtype: object


In [9]:
import pandas as pd

# Desativar truncamento de linhas ao imprimir
pd.set_option("display.max_rows", None)

# Agora imprime todos os dtypes de cada coluna
print(df.dtypes)

# (Opcional) restaurar a configuração padrão
pd.reset_option("display.max_rows")


ID do registro.                                                        int64
Nome do negócio                                                       object
Etapa do negócio                                                      object
Data de fechamento                                                    object
Proprietário do negócio                                               object
Valor                                                                float64
Motivo de Perda                                                       object
Sub-motivo de perda do negócio                                        object
Pré-análise de crédito                                                object
Tipo de Imóvel                                                        object
Dados da fonte de tráfego mais recente 1                              object
Dados da fonte de tráfego mais recente 2                              object
Fonte de tráfego mais recente                                         object

In [10]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# ====================
# 1. CRIAR OU CARREGAR O DATAFRAME
# ====================
# Exemplo: Se você já tem o DataFrame 'df' em memória:
# df = pd.read_csv("seu_arquivo.csv")

# Vamos supor que existam as colunas:
#   "Data de criação" -> datetime ou string
#   "status_cadastro" -> "Aprovado", "Reprovado", etc.
#   "ID do registro." -> Identificador do lead
#   "Foi conectado"   -> "Sim", "Não" ou valores em branco ("")
# Converta para datetime se ainda não estiver:
df["Data de criação"] = pd.to_datetime(df["Data de criação"], errors="coerce")

# ===========================
# 2. INICIALIZAR O APP E O LAYOUT
# ===========================
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard - Leads Aprovados por Semana", style={"textAlign": "center"}),

    # ---------------------------------------------------------
    # FILTROS
    # ---------------------------------------------------------
    html.Div([
        # Filtro 1: Intervalo de Datas
        html.Label("Selecione o intervalo de datas:"),
        dcc.DatePickerRange(
            id="date_range_picker",
            start_date=df["Data de criação"].min(),  # Data inicial padrão
            end_date=df["Data de criação"].max(),    # Data final padrão
            display_format="YYYY-MM-DD"
        ),

        html.Br(),
        html.Br(),

        # Filtro 2: Dropdown para "Foi conectado"
        html.Label("Foi conectado?"),
        dcc.Dropdown(
            id="foi_conectado_dropdown",
            options=[
                {"label": "Sim", "value": "Sim"},
                {"label": "Não", "value": "Não"},
                {"label": "Vazio", "value": ""}  # Se vazio for de fato string vazia
            ],
            value="Sim",  # valor inicial
            clearable=False
        ),
    ], style={"width": "30%", "float": "left", "padding": "20px"}),

    # ---------------------------------------------------------
    # GRÁFICO
    # ---------------------------------------------------------
    html.Div([
        dcc.Graph(id="grafico-leads-aprovados")
    ], style={"width": "65%", "float": "right", "padding": "20px"})
])

# ===========================
# 3. CALLBACK PARA ATUALIZAR O GRÁFICO
# ===========================
@app.callback(
    Output("grafico-leads-aprovados", "figure"),
    [
        Input("date_range_picker", "start_date"),
        Input("date_range_picker", "end_date"),
        Input("foi_conectado_dropdown", "value")
    ]
)
def atualizar_grafico(start_date, end_date, foi_conectado_valor):
    """
    1. Filtrar o DataFrame original (df) usando a data de criação entre 'start_date' e 'end_date'.
    2. Filtrar também pela condição de "Foi conectado" (Sim, Não, ou vazio).
    3. Criar coluna da semana do ano.
    4. Filtrar status_cadastro == "Aprovado".
    5. Agrupar e plotar.
    """
    # Criar uma cópia para filtrar
    df_filtrado = df.copy()

    # Filtrar por datas
    mask_data = (df_filtrado["Data de criação"] >= pd.to_datetime(start_date)) & \
                (df_filtrado["Data de criação"] <= pd.to_datetime(end_date))
    df_filtrado = df_filtrado.loc[mask_data]

    # Filtrar por "Foi conectado"
    # "foi_conectado_valor" pode ser "Sim", "Não" ou ""
    mask_conectado = (df_filtrado["Foi conectado"] == foi_conectado_valor)
    df_filtrado = df_filtrado.loc[mask_conectado]

    # Criar/atualizar coluna de semana do ano
    df_filtrado["semana_do_ano"] = df_filtrado["Data de criação"].dt.isocalendar().week

    # Filtrar somente leads aprovados
    df_aprovados = df_filtrado[df_filtrado["status_cadastro"] == "Aprovado"]

    # Agrupar por semana
    df_agg = df_aprovados.groupby("semana_do_ano")["ID do registro."].count().reset_index()
    df_agg.columns = ["semana_do_ano", "num_leads_aprovados"]

    # Criar figura com plotly
    fig = px.bar(
        df_agg,
        x="semana_do_ano",
        y="num_leads_aprovados",
        labels={"semana_do_ano": "Semana do Ano", "num_leads_aprovados": "Leads Aprovados"},
        title="Leads Aprovados por Semana (Filtrados)"
    )

    return fig

# ===========================
# 4. EXECUTAR O APP
# ===========================
if __name__ == "__main__":
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [45]:
import pandas as pd
import numpy as np

# Exemplo: suponde que já existe o DataFrame 'df'
# com a coluna "Data de criação" em formato datetime
# Se estiver em string, converta:
df["Data de criação"] = pd.to_datetime(df["Data de criação"], errors="coerce")

def verificar_horario_comercial(dt: pd.Timestamp) -> str:
    """
    Retorna 'Sim' se a data/hora estiver dentro do horário comercial,
    caso contrário, retorna 'Não'.

    Horário comercial definido como:
    - Seg a Sex (dayofweek 0 a 4):  08:00 <= hora < 20:00
    - Sáb (dayofweek 5):           08:00 <= hora < 17:00
    - Dom (dayofweek 6):          (fora do horário comercial)
    """
    if pd.isna(dt):
        # Se a data estiver nula (NaT), consideramos como 'Não'
        return "Não"
    
    day_of_week = dt.dayofweek  # Seg=0 ... Dom=6
    hour = dt.hour

    # Segunda a Sexta
    if 0 <= day_of_week <= 4:
        return "Sim" if (8 <= hour < 20) else "Não"
    # Sábado
    elif day_of_week == 5:
        return "Sim" if (8 <= hour < 17) else "Não"
    # Domingo
    else:  # day_of_week == 6
        return "Não"

# Cria a nova coluna "Horario Comercial" (Sim/Não)
df["Horario Comercial"] = df["Data de criação"].apply(verificar_horario_comercial)

# Conferir o resultado
print(df[["Data de criação", "Horario Comercial"]].head())


      Data de criação Horario Comercial
0 2024-12-21 00:17:00               Não
1 2024-12-20 23:31:00               Não
2 2024-12-20 18:02:00               Sim
3 2024-12-20 14:33:00               Sim
4 2024-12-20 09:50:00               Sim


In [47]:
# Nome da coluna
coluna_pre_analise = 'Date entered "Pré-análise crédito (Funil.Vendas)"'

# Verificar se a coluna existe no DataFrame
if coluna_pre_analise in df.columns:
    # Verificar os valores únicos na coluna
    print("Valores únicos na coluna:")
    print(df[coluna_pre_analise].unique())

    # Substituir valores vazios ou inválidos (ex.: strings vazias) por NaN
    df[coluna_pre_analise] = df[coluna_pre_analise].replace("", pd.NA)

    # Filtrar os registros com valores preenchidos
    df_pre_analise = df[df[coluna_pre_analise].notna()]

    # Contar o total de leads que foram para pré-análise
    total_pre_analise = len(df_pre_analise)

    # Exibir o resultado
    print(f"Total de Leads que foram para Pré-análise de Crédito: {total_pre_analise}")

    # Exibir os primeiros valores válidos
    print("Primeiros valores válidos na coluna:")
    print(df_pre_analise[coluna_pre_analise].head(10))
else:
    print(f"A coluna '{coluna_pre_analise}' não existe no DataFrame.")


Valores únicos na coluna:
[nan '2024-12-17 12:10' '2024-12-17 13:01' ... '2024-12-17 13:19'
 '2024-12-12 15:30' '2024-11-28 15:47']
Total de Leads que foram para Pré-análise de Crédito: 1807
Primeiros valores válidos na coluna:
63     2024-12-17 12:10
82     2024-12-17 13:01
155    2024-12-16 18:41
161    2024-12-16 16:27
233    2024-12-17 12:07
239    2024-12-14 14:31
272    2024-12-16 17:40
394    2024-12-11 17:26
442    2024-12-10 19:27
483    2024-12-10 12:06
Name: Date entered "Pré-análise crédito (Funil.Vendas)", dtype: object


In [83]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# ====================
# 1. PREPARAR O DATAFRAME
# ====================
# Certifique-se de que o DataFrame `df` está carregado no ambiente.

# Converter 'Data de criação' para datetime, se necessário
df["Data de criação"] = pd.to_datetime(df["Data de criação"], errors="coerce")

# Criar a coluna 'Horario Comercial'
df["Horario Comercial"] = df["Data de criação"].apply(
    lambda x: (
        "Sim" if (
            x.weekday() < 5 and 8 <= x.hour < 20 or
            x.weekday() == 5 and 8 <= x.hour < 17
        ) else "Não"
    ) if pd.notna(x) else "Indefinido"
)

# Criar a coluna 'Tentativa de Contato'
def classificar_tentativa(row):
    if pd.notna(row['Date entered "Tentativa de contato 1 (Funil.Vendas)"']) and pd.isna(row['Date entered "Tentativa de contato 2 (Funil.Vendas)"']):
        return "Tentativa 1 sem Tentativa 2"
    elif pd.notna(row['Date entered "Tentativa de contato 2 (Funil.Vendas)"']) and pd.isna(row['Date entered "Tentativa de contato 3 (Funil.Vendas)"']):
        return "Tentativa 2 sem Tentativa 3"
    elif pd.notna(row['Date entered "Tentativa de contato 3 (Funil.Vendas)"']):
        return "Três Tentativas"
    else:
        return "Nenhuma Tentativa"

df["Tentativa de Contato"] = df.apply(classificar_tentativa, axis=1)

# ====================
# 2. FUNÇÕES AUXILIARES
# ====================
def calcular_percentual_avanco(df, coluna_avanco, horario_comercial_col):
    """
    Calcula o percentual de avanço por 'Horario Comercial'.
    """
    df = df.copy()
    df["avancou"] = df[coluna_avanco].notna()

    df_agg = df.groupby(horario_comercial_col).agg(
        total_leads=("ID do registro.", "count"),
        avancou=("avancou", "sum")
    ).reset_index()

    df_agg["percentual_avanco"] = df_agg["avancou"] / df_agg["total_leads"] * 100
    return df_agg

# ====================
# 3. CRIAR O DASHBOARD
# ====================
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard - Leads Aprovados", style={"textAlign": "center"}),

    # Filtros
    html.Div([
        html.Label("Intervalo de Datas:"),
        dcc.DatePickerRange(
            id="date-picker-range",
            start_date=df["Data de criação"].min(),
            end_date=df["Data de criação"].max(),
            display_format="YYYY-MM-DD"
        ),

        html.Br(),

        html.Label("Tentativa de Contato:"),
        dcc.Dropdown(
            id="tentativa-dropdown",
            options=[
                {"label": "Todas as Tentativas", "value": "Todas"},
                {"label": "Tentativa 1 sem Tentativa 2", "value": "Tentativa 1 sem Tentativa 2"},
                {"label": "Tentativa 2 sem Tentativa 3", "value": "Tentativa 2 sem Tentativa 3"},
                {"label": "Três Tentativas", "value": "Três Tentativas"}
            ],
            value="Todas",  # Valor padrão
            clearable=False
        )
    ], style={"padding": "20px"}),

    # Gráficos
    html.Div([
        dcc.Graph(id="grafico-leads-aprovados"),
        dcc.Graph(id="grafico-conectados"),
        dcc.Graph(id="grafico-pre-analise"),
        dcc.Graph(id="grafico-analise-credito"),
    ], style={"padding": "20px"})
])

# ====================
# 4. CALLBACKS PARA OS GRÁFICOS
# ====================
@app.callback(
    [Output("grafico-leads-aprovados", "figure"),
     Output("grafico-conectados", "figure"),
     Output("grafico-pre-analise", "figure"),
     Output("grafico-analise-credito", "figure")],
    [Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date"),
     Input("tentativa-dropdown", "value")]
)
def atualizar_graficos(start_date, end_date, tentativa):
    # Filtrar os dados com base nos inputs
    df_filtrado = df[
        (df["Data de criação"] >= pd.to_datetime(start_date)) &
        (df["Data de criação"] <= pd.to_datetime(end_date))
    ]

    if tentativa != "Todas":
        df_filtrado = df_filtrado[df_filtrado["Tentativa de Contato"] == tentativa]

    # Gráfico 1: Leads aprovados por semana
    df_filtrado["semana_do_ano"] = df_filtrado["Data de criação"].dt.isocalendar().week
    df_aprovados = df_filtrado[df_filtrado["status_cadastro"] == "Aprovado"]
    df_agg_semanal = df_aprovados.groupby("semana_do_ano")["ID do registro."].count().reset_index()
    df_agg_semanal.columns = ["semana_do_ano", "num_leads_aprovados"]
    fig_leads_aprovados = px.bar(
        df_agg_semanal,
        x="semana_do_ano",
        y="num_leads_aprovados",
        labels={"semana_do_ano": "Semana do Ano", "num_leads_aprovados": "Leads Aprovados"},
        title="Leads Aprovados por Semana"
    )

    # Gráfico 2: Percentual de leads conectados por horário comercial
    df_aprovados["foi_conectado"] = df_aprovados["Foi conectado"] == "Sim"
    df_agg_conectados = df_aprovados.groupby("Horario Comercial").agg(
        total_leads=("ID do registro.", "count"),
        conectados=("foi_conectado", "sum")
    ).reset_index()
    df_agg_conectados["percentual_conectados"] = df_agg_conectados["conectados"] / df_agg_conectados["total_leads"] * 100
    fig_conectados = px.bar(
        df_agg_conectados,
        x="Horario Comercial",
        y="percentual_conectados",
        title="Percentual de Leads Conectados por Horário Comercial",
        labels={"Horario Comercial": "Horário Comercial", "percentual_conectados": "% Conectados"},
        text="percentual_conectados"
    )
    fig_conectados.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
    fig_conectados.update_layout(yaxis_tickformat=".2f%%")

    # Gráfico 3: Percentual de avanço para pré-análise de crédito
    df_pre_analise = calcular_percentual_avanco(
        df_filtrado,
        coluna_avanco='Date entered "Pré-análise crédito (Funil.Vendas)"',
        horario_comercial_col="Horario Comercial"
    )
    fig_pre_analise = px.bar(
        df_pre_analise,
        x="Horario Comercial",
        y="percentual_avanco",
        title="Percentual de Avanço para Pré-Análise de Crédito",
        labels={"Horario Comercial": "Horário Comercial", "percentual_avanco": "% Avanço"},
        text="percentual_avanco"
    )
    fig_pre_analise.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
    fig_pre_analise.update_layout(yaxis_tickformat=".2f%%")

    # Gráfico 4: Percentual de avanço para análise de crédito
    df_analise_credito = calcular_percentual_avanco(
        df_filtrado,
        coluna_avanco='Date entered "Análise crédito (Funil.Vendas)"',
        horario_comercial_col="Horario Comercial"
    )
    fig_analise_credito = px.bar(
        df_analise_credito,
        x="Horario Comercial",
        y="percentual_avanco",
        title="Percentual de Avanço para Análise de Crédito",
        labels={"Horario Comercial": "Horario Comercial", "percentual_avanco": "% Avanço"},
        text="percentual_avanco"
    )
    fig_analise_credito.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
    fig_analise_credito.update_layout(yaxis_tickformat=".2f%%")

    return fig_leads_aprovados, fig_conectados, fig_pre_analise, fig_analise_credito

# ====================
# 5. EXECUTAR O APP
# ====================
if __name__ == "__main__":
    app.run_server(debug=True)


In [11]:
pip install Dash


Note: you may need to restart the kernel to use updated packages.
